# ENCODE Long Read Sequel2 dataset

This tutorial guides through a typical Long Read Transcriptome Sequencing (LRTS) analysis workflow with isotools, 
using ENCODE Isoseq SequelII Data. It demonstrates the analysis of alternative splicing events within and between sample groups. 

## Preparation
Select the "long read RNA-seq" samples in the ENCODE data portal (https://www.encodeproject.org/) and download aligned .bam files. Here I use all Sequel II samples available at the time of writing, but you choose to process a subset.
Download the files with curl and use samtools to index the bam files.
``` sh
xargs -L 1 curl -O -J -L < ENCODE_files.txt
for bam in *bam; 
    do samtools index $bam
done
```
In addition to the bamfiles, a file metadata.tsv should have been downloaded, which we use to read the sample information. Further you need a reference annotation and a genome fastq file. Please see the Alzheimer tutorial how to get these files. 

## Data Import

In [1]:
from  isotools import Transcriptome
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import logging
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


In [5]:
#I assume the encode data is in an subdirctory "encode"

#first, process the metadata
metadata=pd.read_csv('encode/metadata.tsv', sep='\t')
platform={r['Experiment accession']: r['Platform'] for i,r in metadata[~ metadata.Platform.isnull()].iterrows()}

#Select the samples from the metadata (here all SequelII files), make sure there are only bam files are in the table
samples=metadata[(metadata['Output type']=='alignments') & (metadata['File Status']=='released')].copy()
samples['Platform']=[platform[ea] for ea in samples['Experiment accession']] #This info is missing for some files
samples=samples[['File accession','Output type','Biosample term name','Biosample type','Technical replicate(s)','Platform']].reset_index(drop=True) 
samples=samples[samples.Platform=='Pacific Biosciences Sequel II'].reset_index(drop=True) 


chrom=[f'chr{i+1}'for i in range(22)]+['chrX','chrY']
#ENCODE uses an interesting reference, including sequences from viruses
genome='reference/GRCh38_genome_primary_assembly_gencode_v29.fa'
anno='reference/gencode.v36.chr_patch_hapl_scaff.annotation_sorted'

#print the sample table
samples

,File accession,Output type,Biosample term name,Biosample type,Technical replicate(s),Platform
0,ENCFF745HHL,alignments,aorta,tissue,1_1,Pacific Biosciences Sequel II
1,ENCFF509GHY,alignments,progenitor cell of endocrine pancreas,in vitro differentiated cells,2_1,Pacific Biosciences Sequel II
2,ENCFF803KIA,alignments,progenitor cell of endocrine pancreas,in vitro differentiated cells,1_1,Pacific Biosciences Sequel II
3,ENCFF971JDY,alignments,mesenteric fat pad,tissue,1_1,Pacific Biosciences Sequel II
4,ENCFF292UIE,alignments,H9,cell line,1_1,Pacific Biosciences Sequel II
5,ENCFF738RAA,alignments,H9,cell line,2_1,Pacific Biosciences Sequel II
6,ENCFF911RNV,alignments,right cardiac atrium,tissue,1_1,Pacific Biosciences Sequel II
7,ENCFF472TSL,alignments,posterior vena cava,tissue,1_1,Pacific Biosciences Sequel II
8,ENCFF319JFG,alignments,neural crest cell,in vitro differentiated cells,1_1,Pacific Biosciences Sequel II
9,ENCFF901XCR,alignments,neural crest cell,in vitro differentiated cells,2_1,Pacific Biosciences Sequel II


In [ ]:
try:    
    isoseq=Transcriptome(f'encode/encode_isotools.pkl')
except FileNotFoundError:
    try:
        isoseq=Transcriptome.from_reference(anno+'.isotools.pkl')
    except FileNotFoundError:
        isoseq=Transcriptome.from_reference(anno+'.gff3.gz')
        isoseq.save_reference(anno+'.isotools.pkl')
    for i,row in samples.iterrows():
        sname=f"{row['Biosample term name']}_{row['Technical replicate(s)']}".replace(' ','_')
        if sname in isoseq.samples:
            sname+='_b'
        isoseq.add_sample_from_bam(f"encode/{row['File accession']}.bam", 
                                   sample_name=sname , 
                                   group=row['Biosample term name'] ,
                                   sample_type=row['Biosample type'],
                                   platform=row['Platform']) 
    isoseq.add_biases(genome_fn) #3:41 min
    isoseq.make_index()
    isoseq.save('encode/encode_isotools.pkl')


INFO:loading transcriptome from encode/encode_isotools.pkl
INFO:importing reference from pkl file reference/gencode.v36.chr_patch_hapl_scaff.annotation_sorted.isotools.pkl
INFO:adding sample aorta_1_1
100%|██████████| 1960872.0/1960872 [07:11<00:00, 4545.35reads/s, chr=ERCC-00171]             
INFO:adding sample progenitor_cell_of_endocrine_pancreas_2_1
100%|██████████| 1167968.0/1167968 [04:05<00:00, 4758.71reads/s, chr=ERCC-00171]             
INFO:adding sample progenitor_cell_of_endocrine_pancreas_1_1
100%|██████████| 1623139.0/1623139 [05:44<00:00, 4708.39reads/s, chr=ERCC-00171]             
INFO:adding sample mesenteric_fat_pad_1_1
100%|██████████| 1816266.0/1816266 [04:50<00:00, 6253.37reads/s, chr=ERCC-00171]             
INFO:adding sample H9_1_1
100%|██████████| 3201257.0/3201257 [10:35<00:00, 5036.39reads/s, chr=ERCC-00171]             
INFO:adding sample H9_2_1
100%|██████████| 2753910.0/2753910 [08:37<00:00, 5325.41reads/s, chr=ERCC-00171]             
INFO:adding sample 

## Quality control and filtering
First, we will depict some quality metrics and define filtering criteria.

In [ ]:
#add gencode specific filters
ref_filter={'UNSPLICED':'len(exons)==1','HIGH_SUPPORT':'transcript_support_level=="1"', 'PROTEIN_CODING':'transcript_type=="protein_coding"'}
#encode bams do not have chimeric redas 
isoseq.add_filter( gene_filter=gene_filter, ref_transcript_filter=ref_filter)


In [ ]:
#compute some summary statistics on technical artifacts. 
# For this analysis samples are grouped by 'Biosample term name'
tr_stats=[
    isoseq.transcript_length_hist(groups=isoseq.groups(), add_reference=True, min_coverage=2,tr_filter=dict( remove=['NOVEL_GENE']), ref_filter=dict(include=['HIGH_SUPPORT'])),
    isoseq.downstream_a_hist(groups=isoseq.groups(), tr_filter=dict( remove=['NOVEL_GENE', 'UNSPLICED']), ref_filter=dict(remove=['UNSPLICED'])),
    isoseq.downstream_a_hist(groups=isoseq.groups(), tr_filter=dict(include=['NOVEL_GENE', 'UNSPLICED'])),
    isoseq.direct_repeat_hist(groups=isoseq.groups(),bins=np.linspace(-.5,10.5,12))]

tr_stats.append((pd.concat([tr_stats[2][0].add_suffix(' novel unspliced'),tr_stats[1][0].add_suffix(' known multiexon')],axis=1),tr_stats[2][1]))

#statistic on the filter flags
f_stats=isoseq.filter_stats( groups=isoseq.groups(), weight_by_coverage=True,min_coverage=1)


## Data Exploration
To explore the relation of the samples with respect to splicing we look at PCA and UMAP embeddings based on alternative splicing events.

In [ ]:
#Compute alternative splicing events by finding "bubbles" in the segment graph
splice_events=isoseq.find_splice_bubbles()

In [ ]:
from isotools.plots import plot_embedding
plt.rcParams["figure.figsize"] = (20,10)
components={}
pca={}
p={}
f,axs=plt.subplots(2,4)
for ax,t in zip(axs.flatten(),['all','3AS','5AS','ES','IR','ME', 'TSS', 'PAS']):
    p_i,comp_i, pca_i=plot_embedding(splice_events, ax=ax, labels=True, groups=isoseq.groups(), splice_types=t)#, colors={'mut':'red', 'wt':'green'})
    components[t]=comp_i
    pca[t]=pca_i
    p[t]=p_i

axs[0,2].legend(fontsize='medium', ncol=1,handleheight=2.4, labelspacing=0.05, bbox_to_anchor=(1.02, 1), loc='upper left')
plt.tight_layout()

plt.savefig(out_fn+'_PCA.png') 

